In [ ]:
%matplotlib notebook
import os
import matplotlib.pyplot as plt
import numpy as np
import firedrake
import icepack, icepack.plot, icepack.models

# Jakobshavn, the Beast

In this demo we'll study the fastest glacier on earth: Jakobshavn Isbrae, or Sermeq Kujalleq if you prefer the Greenlandic name.
We'll tread over some familiar territory, albeit in an unfamiliar way.
To initialize the simulation, we'll estimate the friction coefficient using the shallow stream equations.
But for the fact that we're using a slightly more sophisticated physics model and inferring the friction instead of the rheology, this should all be familiar from demo \#4.

Next we'll lift everything up into 3D and study how heat is transported through a glacier.
The physical effect that we want to demonstrate is how strain heating softens the margins.
The field of interest here is the *internal energy density* $E$, which has units of MPa / m${}^3$.
The internal energy density encompasses both the ice temperature and the latent heat stored in meltwater.
For a given temperature $T$ and meltwater fraction $f$, the internal energy density is
$$E = \rho (c_pT + Lf)$$
where $c_p$ is the specific heat capacity of ice at constant pressure and $L$ is the latent heat of melting.
An englacial meltwater fraction of 0.33% is on the high side.
The energy density can be a much more convenient field to work with than the temperature because we don't have to track the moving cold-temperate transition surface.

Thermal energy is transported by two mechanisms: advection and vertical diffusion.
Horizontal diffusion is completely negligible.
The main sources of thermal energy are geothermal heating from the ice base, exchange of heat with the atmosphere at the ice surface, and strain heating from within.
Putting all this together, the equations of heat transport are:
$$\frac{\partial E}{\partial t} + \nabla\cdot Eu - \frac{\partial}{\partial z}\alpha\frac{\partial E}{\partial z} = \tau : \dot\varepsilon,$$
with the boundary condition
$$-Ew + \alpha\frac{\partial E}{\partial z} = q_b$$
at the ice base.

### Input data

First, we'll load in a coarse mesh of the domain.
This mesh won't have the kind of resolution that we need to capture the sharp thickness gradients that occur near the Jakobshavn trough.
Rather than refine the geometry using gmsh, I'll show you how to get a uniform refinement of a domain entirely from within firedrake.

In [ ]:
data_directory = os.environ['ICEPACK_DATA']

In [ ]:
mesh_coarse = firedrake.Mesh(os.path.join(data_directory, 'meshes/jakobshavn/jakobshavn.msh'))

fig, axes = icepack.plot.subplots()
axes.set_xlabel('meters')
axes.grid()
icepack.plot.triplot(mesh_coarse, axes=axes, linewidth=2)
plt.show(fig)

The firedrake function `MeshHierarchy` will generate a sequence of uniformly refined meshes from an initial coarse mesh.
Mesh hierarchies make it easy to implement the geometric multigrid method, but we won't discuss that here.
Instead, we'll just take the last element of the list.

In [ ]:
mesh_hierarchy = firedrake.MeshHierarchy(mesh_coarse, refinement_levels=1)
mesh = mesh_hierarchy[-1]

fig, axes = icepack.plot.subplots()
axes.set_xlabel('meters')
axes.grid()
icepack.plot.triplot(mesh, axes=axes, linewidth=2)
plt.show(fig)

Reading in and interpolating gridded data works just like before.

In [ ]:
from icepack.grid import arcinfo
surface = arcinfo.read(os.path.join(data_directory, 'bedmachine_greenland/jakobshavn-surface.txt'))
bed = arcinfo.read(os.path.join(data_directory, 'bedmachine_greenland/jakobshavn-bed.txt'))
vx = arcinfo.read(os.path.join(data_directory, 'measures_greenland/annual/jakobshavn-vx.txt'))
vy = arcinfo.read(os.path.join(data_directory, 'measures_greenland/annual/jakobshavn-vy.txt'))

In [ ]:
degree = 2
Q = firedrake.FunctionSpace(mesh, family='CG', degree=degree)
V = firedrake.VectorFunctionSpace(mesh, family='CG', degree=degree)

s = icepack.interpolate(surface, Q)
b = icepack.interpolate(bed, Q)
h = icepack.interpolate(s - b, Q)
u_obs = icepack.interpolate((vx, vy), V)

In [ ]:
fig, axes = icepack.plot.subplots()
contours = icepack.plot.tricontourf(u_obs, 40, cmap='magma', axes=axes)
colorbar = fig.colorbar(contours)
colorbar.set_label('m / year')
plt.show(fig)

Jakobshavn flows through a massive trench that goes from a depth of just above sea level to 1.5km below, all in a space of less 5km.
It's likely that the glacier itself gouged out this feature from the bedrock through millenia of erosion.
We'll be using simplified flow models in this demo for illustrative purposes, but the relief here is so extreme that even the assumption of a low aspect ratio is questionable.
If you were doing "serious" work you'd probably want to use the full Stokes equations.

In [ ]:
fig, axes = icepack.plot.subplots()
contours = icepack.plot.tricontourf(b, 40, cmap='magma', axes=axes)
colorbar = fig.colorbar(contours)
plt.show(fig)

### Estimating basal friction

The heat generated at the ice base from sliding is a key input to the thermal transport equation, but we have no a priori measurements of sliding friction.
We can estimate a sensible value using inverse methods, much like we did for the rheology of the Larsen Ice Shelf.
Unfortunately, for grounded glaciers things get really circular really quick -- we need a basal friction to estimate temperature, but we need a temperature to estimate basal friction.
A single snapshot measurement of the glacier thickness and velocity is not enough to infer both at the same time.

As a first guess for the friction coefficient, we'll assume that the bed supports most of the driving stress, while the rest is taken up by longitudinal extension.
The driving stress is proportional to the surface slope, and calculating this directly amounts to differentiating a noisy data set.
To avoid any artifacts we'll apply a smoothing filter first.

In [ ]:
from icepack.constants import rho_ice as ρ_I, gravity as g
from firedrake import inner, grad, dx
f = -ρ_I * g * h * inner(grad(s), u_obs) / firedrake.sqrt(inner(u_obs, u_obs))
τ_d = firedrake.Function(Q)
L = 5e3
J = 0.5 * ((τ_d - f)**2 + L**2 * inner(grad(τ_d), grad(τ_d))) * dx
firedrake.solve(firedrake.derivative(J, τ_d) == 0, τ_d,
                solver_parameters={'ksp_type': 'preonly', 'pc_type': 'lu'})

The component of the driving stress along the ice velocity is plotted below.
Part of the reason Jakobshavn flows so fast is that the surface slopes are so high -- the driving stresses are on the order of 200 kPa.
Compare this to the ice streams feeding the Ross Ice Shelf, where the driving stress is as low as 20-40 kPa.

In [ ]:
fig, axes = icepack.plot.subplots()
contours = icepack.plot.tricontourf(τ_d, 40, cmap='magma', axes=axes)
colorbar = fig.colorbar(contours)
colorbar.set_label('MPa')
plt.show(fig)

Now that we've got a stress to match, we'll compute an initial value of the friction coefficient.
We'll assume to start that bed friction takes up 90% of the driving stress.
This is a good approximation outside the main trunk of the glacier, but a poor one in fast-flowing areas, as we'll soon see.
We also need to use a parameterization of the basal friction that guarantees positivity, much like how we did with the rheology.

In [ ]:
from icepack.constants import glen_flow_law as n
import icepack.models.friction

τ_0, u_0 = 0.1, 100.0
def bed_friction(u, β):
    C = τ_0 / u_0**(1/n) * firedrake.exp(β)
    return icepack.models.friction.bed_friction(u, C)

C = 0.9 * τ_d / firedrake.sqrt(inner(u_obs, u_obs))**(1/n)
β = firedrake.interpolate(firedrake.ln(u_0**(1/n) * C / τ_0), Q)

As a blunt initial guess, we'll take the ice temperature to be a uniform -20${}^\circ$C.
This is very blunt; we'll go back and revisit it later.

In [ ]:
A = firedrake.Constant(icepack.rate_factor(253.0))

As a sanity check, let's make sure that the computed velocity isn't absurdly far off from the observations.

In [ ]:
model = icepack.models.IceStream(friction=bed_friction)
opts = {'dirichlet_ids': [2], 'side_wall_ids': [3, 4], 'tol': 1e-8}
u = model.diagnostic_solve(u0=u_obs, h=h, s=s, β=β, A=A, **opts)

The max speed is vaguely close to that of the observations, but the region of fast flow is much smaller than the real glacier.

In [ ]:
fig, axes = icepack.plot.subplots()
contours = icepack.plot.tricontourf(u, 40, cmap='magma', axes=axes)
colorbar = fig.colorbar(contours)
colorbar.set_label('m / year')
plt.show(fig)

The setup for the inverse problem is similar to what we did for the rheology.